In [2]:
%env kubectl=/usr/bin/kubectl
%env istio=/Users/gsjurseth/istio-1.0.2
%env istioctl=/Users/gsjurseth/istio-1.0.2/bin/istioctl
%env ingress=35.226.214.129
%env demo=/Users/gsjurseth/dev/fazio/microservices-demo

env: kubectl=/usr/bin/kubectl
env: istio=/Users/gsjurseth/istio-1.0.2
env: istioctl=/Users/gsjurseth/istio-1.0.2/bin/istioctl
env: ingress=35.226.214.129
env: demo=/Users/gsjurseth/dev/fazio/microservices-demo


---
## Fazio Retail Shop UI

http://35.226.214.129/


Let's look at the running pods

In [3]:
!$kubectl get pods

NAME                                     READY   STATUS    RESTARTS   AGE
adservice-6578bcfd8f-rcdwj               2/2     Running   0          12d
cartservice-5cb655594d-x4w77             2/2     Running   423        12d
checkoutservice-799869764-ztndz          2/2     Running   0          12d
currencyservice-6cbdc7768d-n8jw6         2/2     Running   0          12d
emailservice-9b7867fcb-xflr2             2/2     Running   0          12d
frontend-blue-6fb5d8d967-lbmw7           2/2     Running   0          3m
frontend-gray-5447f56f9b-b7qng           2/2     Running   0          3m
loadgenerator-cf64757b5-vr9db            2/2     Running   0          12d
paymentservice-57cd6d499f-jddb9          2/2     Running   0          12d
productcatalogservice-584d6ff79d-kn2xr   2/2     Running   0          12d
recommendationservice-69869d97b9-pkv99   2/2     Running   0          12d
redis-cart-9b864d47f-hqrgr               2/2     Running   0          12d
shippingservice-7b7759d995-5vzn5        

---
## Architectural Diagram

![architecture](https://raw.githubusercontent.com/theganyo/microservices-demo/fazio-playground/img/architecture-diagram.jpeg)

---
## I've heard that blue is hip color!

But is it really? Let's canary test a blue front-end. We'll route 20% of traffic to the blue version.

In [10]:
%%bash
cat <<EOF | $kubectl apply -f -

apiVersion: networking.istio.io/v1alpha3
kind: VirtualService
metadata:
  name: frontend-ingress
spec:
  gateways:
  - frontend-gateway
  hosts:
  - '*'
  http:
  - route:
    - destination:
        host: frontend
        subset: gray
      weight: 50
    - destination:
        host: frontend
        subset: blue
      weight: 50

EOF

virtualservice.networking.istio.io/frontend-ingress configured


---
## It turns out, customers prefer grey.

Let's revert that change.

In [11]:
%%bash
cat <<EOF | $kubectl apply -f -

apiVersion: networking.istio.io/v1alpha3
kind: VirtualService
metadata:
  name: frontend-ingress
spec:
  gateways:
  - frontend-gateway
  hosts:
  - '*'
  http:
  - route:
    - destination:
        host: frontend
        subset: gray
      weight: 100

EOF

virtualservice.networking.istio.io/frontend-ingress configured


---
## Jaeger UI

Run in console:

    kubectl port-forward -n istio-system $(kubectl get pod -n istio-system -l app=jaeger -o jsonpath='{.items[0].metadata.name}') 16686:16686 &
    
http://localhost:16686/search?service=emailservice

---
## Grafana UI

Run in console:

    kubectl -n istio-system port-forward $(kubectl -n istio-system get pod -l app=grafana -o jsonpath='{.items[0].metadata.name}') 3000:3000 &
    
http://localhost:3000/d/3/pilot-dashboard